# MLX Experiments — Hands-On LLM Exploration

**Hardware:** M3 Pro, 36 GB unified memory  
**Goal:** See every concept from our React app with REAL models  

Run each cell, read the comments, observe the output. Tweak and re-run!

---
## Setup

Run this once. If you already installed, skip to Experiment 1.

In [ ]:
# Install dependencies (run once)
!pip install mlx mlx-lm transformers huggingface_hub

In [ ]:
# Verify installation
import mlx.core as mx
print(f"MLX version: {mx.__version__}")
print(f"Backend: {mx.default_device()}")

import mlx_lm
print(f"MLX-LM installed ✓")

---
## Download Models (One-Time)

Downloads models to `./models/` so they survive cache cleanups. Run once — skip after that.

In [ ]:
from huggingface_hub import snapshot_download
import os

MODELS_DIR = os.path.join(os.path.dirname(os.path.abspath("__file__")), "models")

# Download Qwen 8B (~5 GB) — our primary model
qwen_path = os.path.join(MODELS_DIR, "Qwen3-8B-4bit")
if not os.path.exists(qwen_path):
    print("Downloading Qwen3-8B-4bit (~5 GB)... grab a coffee ☕")
    snapshot_download(
        repo_id="mlx-community/Qwen3-8B-4bit",
        local_dir=qwen_path,
    )
    print(f"Saved to: {qwen_path}")
else:
    print(f"Qwen3-8B-4bit already downloaded at {qwen_path}")

# Download Llama 3B (~2 GB) — for comparison
llama_path = os.path.join(MODELS_DIR, "Llama-3.2-3B-Instruct-4bit")
if not os.path.exists(llama_path):
    print("Downloading Llama-3.2-3B-Instruct-4bit (~2 GB)...")
    snapshot_download(
        repo_id="mlx-community/Llama-3.2-3B-Instruct-4bit",
        local_dir=llama_path,
    )
    print(f"Saved to: {llama_path}")
else:
    print(f"Llama-3.2-3B-Instruct-4bit already downloaded at {llama_path}")

print("\nBoth models ready! ✓")

---
## Experiment 1: Your First Local LLM

**Goal:** See a model running on YOUR machine. No cloud, no API.

Models are loaded from `./models/` — no internet needed after download.

In [ ]:
from mlx_lm import load, generate

# Load from local path (no internet needed)
model, tokenizer = load(qwen_path)
print("Qwen 8B loaded! ✓")

In [ ]:
# Generate your first response!
# Note: Qwen 3 has a "thinking" mode — it reasons internally before answering.
# Adding /no_think to the prompt disables it for faster, direct answers.

response = generate(
    model,
    tokenizer,
    prompt="What is Rust programming language in one sentence? /no_think",
    max_tokens=100,
    verbose=True
)
print("\n" + response)

In [ ]:
# Now try WITH thinking mode — same question, no /no_think
# You'll see the model "reason" internally before answering.
# It needs more tokens because the thinking eats into the budget.

response = generate(
    model,
    tokenizer,
    prompt="What is Rust programming language in one sentence?",
    max_tokens=500,  # more room for thinking + answer
    verbose=True
)
print("\n" + response)
# Look for the <think>...</think> section — that's the model reasoning!

---
## Experiment 1b: 3B vs 8B — Head to Head

Both models get the **exact same prompts**. We measure:
- **Speed** (tokens/sec) — how fast each generates
- **Quality** — you judge which answer is better  
- **Memory** — how much RAM each uses

This is how you'd pick a model for Jarvis in the real world.

In [ ]:
# Load the small model — measure load time for both
from mlx.utils import tree_flatten
import time

def count_params(m):
    return sum(v.size for _, v in tree_flatten(m.parameters()))

# Re-load 8B to measure load time (already in memory, but we time it)
start = time.time()
model, tokenizer = load(qwen_path)
load_time_8b = time.time() - start
print(f"Qwen 8B loaded in {load_time_8b:.1f}s ✓")

start = time.time()
model_small, tok_small = load(llama_path)
load_time_3b = time.time() - start
print(f"Llama 3B loaded in {load_time_3b:.1f}s ✓")

# Architecture comparison
params_8b = count_params(model)
params_3b = count_params(model_small)
layers_8b = len(model.model.layers)
layers_3b = len(model_small.model.layers)

print(f"\n{'':30s} {'Qwen 8B':>12s}  {'Llama 3B':>12s}")
print("-" * 58)
print(f"{'Parameters':30s} {params_8b:>12,}  {params_3b:>12,}")
print(f"{'Layers':30s} {layers_8b:>12}  {layers_3b:>12}")
print(f"{'Embedding dims':30s} {model.model.embed_tokens.weight.shape[1]:>12,}  {model_small.model.embed_tokens.weight.shape[1]:>12,}")
print(f"{'Vocab size':30s} {model.model.embed_tokens.weight.shape[0]:>12,}  {model_small.model.embed_tokens.weight.shape[0]:>12,}")
print(f"{'Model load time':30s} {load_time_8b:>11.1f}s  {load_time_3b:>11.1f}s")
print(f"{'Est. RAM (Q4)':30s} {'~5.0 GB':>12s}  {'~2.0 GB':>12s}")
print(f"\n→ 8B has ~{params_8b/params_3b:.1f}× more parameters = more 'knowledge' baked into the weights.")

In [ ]:
import time

# ── Benchmark: same prompts, both models, measure everything ──

test_prompts = [
    {
        "name": "Simple fact",
        "prompt": "What is the capital of Japan? Answer in one sentence. /no_think",
        "max_tokens": 50,
    },
    {
        "name": "Summarization",
        "prompt": "Summarize in 2 bullet points: Transformers use self-attention to process all tokens in parallel. Unlike RNNs, they don't need to process tokens sequentially. The attention mechanism lets each token look at every other token, which makes training very efficient on GPUs. /no_think",
        "max_tokens": 100,
    },
    {
        "name": "Reasoning",
        "prompt": "If a train travels 120 km in 2 hours, and then 90 km in 1.5 hours, what was its average speed for the entire journey? Show your work briefly. /no_think",
        "max_tokens": 150,
    },
    {
        "name": "Code explanation",
        "prompt": "Explain what this Python code does: sorted(set(words), key=lambda w: len(w), reverse=True)[:5] /no_think",
        "max_tokens": 100,
    },
    {
        "name": "Creative writing",
        "prompt": "Write a 2-sentence horror story about a smart home assistant. /no_think",
        "max_tokens": 80,
    },
]

models_to_test = [
    ("Qwen 8B", model, tokenizer),
    ("Llama 3B", model_small, tok_small),
]

# Store results for summary table
results = []

for test in test_prompts:
    print(f"\n{'='*70}")
    print(f"TEST: {test['name']}")
    print(f"PROMPT: {test['prompt'][:80]}...")
    print(f"{'='*70}")
    
    for model_name, m, tok in models_to_test:
        start = time.time()
        response = generate(m, tok, prompt=test["prompt"], max_tokens=test["max_tokens"], verbose=False)
        elapsed = time.time() - start
        
        prompt_tokens = len(tok.encode(test["prompt"]))
        out_tokens = len(tok.encode(response))
        tps = out_tokens / elapsed if elapsed > 0 else 0
        
        results.append({
            "test": test["name"],
            "model": model_name,
            "prompt_tokens": prompt_tokens,
            "output_tokens": out_tokens,
            "time": elapsed,
            "tps": tps,
            "response": response.strip(),
        })
        
        print(f"\n  [{model_name}] ({out_tokens} tokens, {tps:.1f} tok/s, {elapsed:.1f}s)")
        print(f"  {response.strip()[:300]}")


# ── Per-test comparison table ──
print(f"\n\n{'='*80}")
print(f"  RESULTS: Per-Test Comparison")
print(f"{'='*80}")
print(f"\n{'TEST':<18} {'MODEL':<12} {'IN TOK':>6} {'OUT TOK':>7} {'TIME':>7} {'TOK/S':>7}")
print("-" * 62)
for r in results:
    print(f"{r['test']:<18} {r['model']:<12} {r['prompt_tokens']:>6} {r['output_tokens']:>7} {r['time']:>6.1f}s {r['tps']:>6.1f}")


# ── Aggregated comparison table ──
print(f"\n\n{'='*80}")
print(f"  RESULTS: Aggregated Comparison")
print(f"{'='*80}")

print(f"\n{'METRIC':<35} {'Qwen 8B':>12} {'Llama 3B':>12} {'Winner':>10}")
print("-" * 72)

for model_name in ["Qwen 8B", "Llama 3B"]:
    locals()[f"r_{model_name.split()[0].lower()}"] = [r for r in results if r["model"] == model_name]

r8 = [r for r in results if r["model"] == "Qwen 8B"]
r3 = [r for r in results if r["model"] == "Llama 3B"]

# Load time
print(f"{'Model load time':<35} {load_time_8b:>11.1f}s {load_time_3b:>11.1f}s {'3B':>10}")

# Avg generation time
avg_time_8b = sum(r["time"] for r in r8) / len(r8)
avg_time_3b = sum(r["time"] for r in r3) / len(r3)
print(f"{'Avg generation time':<35} {avg_time_8b:>11.1f}s {avg_time_3b:>11.1f}s {'3B' if avg_time_3b < avg_time_8b else '8B':>10}")

# Total generation time
total_time_8b = sum(r["time"] for r in r8)
total_time_3b = sum(r["time"] for r in r3)
print(f"{'Total time (all 5 tests)':<35} {total_time_8b:>11.1f}s {total_time_3b:>11.1f}s {'3B' if total_time_3b < total_time_8b else '8B':>10}")

# Avg tokens/sec
avg_tps_8b = sum(r["tps"] for r in r8) / len(r8)
avg_tps_3b = sum(r["tps"] for r in r3) / len(r3)
print(f"{'Avg speed (tok/s)':<35} {avg_tps_8b:>11.1f} {avg_tps_3b:>11.1f} {'3B' if avg_tps_3b > avg_tps_8b else '8B':>10}")

# Total tokens produced
total_tok_8b = sum(r["output_tokens"] for r in r8)
total_tok_3b = sum(r["output_tokens"] for r in r3)
print(f"{'Total tokens produced':<35} {total_tok_8b:>12} {total_tok_3b:>12} {'—':>10}")

# Avg tokens per response
avg_tok_8b = total_tok_8b / len(r8)
avg_tok_3b = total_tok_3b / len(r3)
print(f"{'Avg tokens per response':<35} {avg_tok_8b:>11.1f} {avg_tok_3b:>11.1f} {'—':>10}")

# Parameters
print(f"{'Parameters':<35} {params_8b:>12,} {params_3b:>12,} {'—':>10}")

# RAM
print(f"{'Est. RAM usage (Q4)':<35} {'~5.0 GB':>12} {'~2.0 GB':>12} {'3B':>10}")

# Speed advantage
speed_ratio = avg_tps_3b / avg_tps_8b if avg_tps_8b > 0 else 0
print(f"\n  Speed advantage:  3B is ~{speed_ratio:.1f}× faster")
print(f"  Size advantage:   3B uses ~{5.0/2.0:.1f}× less RAM")
print(f"  Quality:          YOU judge from the side-by-side below ↓")

print(f"\nDone! Run next cells for side-by-side quality comparison.")

In [ ]:
# ── Summary Table ──

print(f"\n{'TEST':<18} {'MODEL':<12} {'TOKENS':>6} {'TIME':>7} {'TOK/S':>7}")
print("-" * 55)

for r in results:
    print(f"{r['test']:<18} {r['model']:<12} {r['tokens']:>6} {r['time']:>6.1f}s {r['tps']:>6.1f}")

# Averages
for model_name in ["Qwen 8B", "Llama 3B"]:
    model_results = [r for r in results if r["model"] == model_name]
    avg_tps = sum(r["tps"] for r in model_results) / len(model_results)
    avg_time = sum(r["time"] for r in model_results) / len(model_results)
    total_tokens = sum(r["tokens"] for r in model_results)
    print(f"\n  {model_name} average: {avg_tps:.1f} tok/s, {avg_time:.1f}s per response, {total_tokens} total tokens")

In [ ]:
# ── Side-by-side responses — read and judge quality yourself ──

print("SIDE-BY-SIDE: Read both responses and judge which is better.\n")
print("Score each pair: 8B wins / tie / 3B wins\n")

test_names = list(dict.fromkeys(r["test"] for r in results))  # unique, ordered

for test_name in test_names:
    pair = [r for r in results if r["test"] == test_name]
    r8b = next(r for r in pair if r["model"] == "Qwen 8B")
    r3b = next(r for r in pair if r["model"] == "Llama 3B")
    
    print(f"{'='*70}")
    print(f"  {test_name.upper()}")
    print(f"{'='*70}")
    print(f"\n  Qwen 8B ({r8b['tps']:.0f} tok/s):")
    print(f"  {r8b['response'][:400]}")
    print(f"\n  Llama 3B ({r3b['tps']:.0f} tok/s):")
    print(f"  {r3b['response'][:400]}")
    
    speed_winner = "3B" if r3b["tps"] > r8b["tps"] else "8B"
    speed_diff = abs(r3b["tps"] - r8b["tps"]) / min(r3b["tps"], r8b["tps"]) * 100
    print(f"\n  Speed winner: {speed_winner} ({speed_diff:.0f}% faster)")
    print()

print("\n→ Typical pattern: 3B is faster, 8B gives better quality.")
print("→ For Jarvis: use 8B for important tasks (summarize, extract),")
print("  3B for quick/simple tasks (short replies, classification).")

In [ ]:
# ── Memory comparison ──
import subprocess, os

def get_process_memory_mb():
    """Get current process RSS in MB (macOS)."""
    result = subprocess.run(
        ["ps", "-o", "rss=", "-p", str(os.getpid())],
        capture_output=True, text=True
    )
    return int(result.stdout.strip()) / 1024

mem_now = get_process_memory_mb()

# Rough estimates based on model file sizes
print("Memory Comparison")
print("=" * 50)
print(f"{'Qwen 8B (Q4)':<25} ~5.0 GB weights in RAM")
print(f"{'Llama 3B (Q4)':<25} ~2.0 GB weights in RAM")
print(f"{'Both loaded (current)':<25} ~{mem_now/1024:.1f} GB process RSS")
print()
print("For Jarvis (single model):")
print(f"  8B only: ~5 GB + OS overhead → ~8 GB total")
print(f"  3B only: ~2 GB + OS overhead → ~5 GB total")
print(f"  Your RAM: 36 GB → plenty of room either way")
print()
print("→ Memory isn't the bottleneck on 36 GB.")
print("→ The real question is: quality vs speed.")

---
## Experiment 2: See the Tokenizer

**Goal:** Use a REAL tokenizer (not our mock one). See how it actually splits text.

Remember from our React app: tokenizer splits text → assigns IDs → embeddings look up those IDs.

In [ ]:
# Tokenize a sentence
text = "Rust is a programming language"
token_ids = tokenizer.encode(text)

print(f"Text: '{text}'")
print(f"Token IDs: {token_ids}")
print(f"Number of tokens: {len(token_ids)}")
print()

# Decode each token to see the pieces
for i, tid in enumerate(token_ids):
    piece = tokenizer.decode([tid])
    print(f"  Token {i}: ID {tid:>6} → '{piece}'")

In [ ]:
# Tokenizer surprises — run this and see what's unexpected!
tests = [
    "Hello",             # one token? or two?
    "hello",             # lowercase — same token?
    "Rust",              # capital R
    "rust",              # lowercase r  
    "tokenization",      # long word — how many pieces?
    "MLX",               # acronym
    "🔥",                # emoji
    "   spaces   ",      # whitespace
    "don't",             # apostrophe
    "ChatGPT",           # compound word
]

print(f"{'Input':<20} {'Tokens':>3}  Pieces")
print("-" * 60)
for t in tests:
    ids = tokenizer.encode(t)
    pieces = [tokenizer.decode([i]) for i in ids]
    print(f"'{t}'" + " " * (18 - len(t)) + f"{len(ids):>3}  {pieces}")

In [ ]:
# KEY INSIGHT: same word, same token ID — tokenizer has NO idea about meaning
# "Rust" the language and "Rust" the corrosion get the SAME token ID

text1 = "Rust is a programming language"
text2 = "Rust on the iron door needs cleaned"

ids1 = tokenizer.encode(text1)
ids2 = tokenizer.encode(text2)

print(f"'{text1}' → first token ID: {ids1[0]}")
print(f"'{text2}' → first token ID: {ids2[0]}")
print(f"\nSame ID? {ids1[0] == ids2[0]}")
print("\n→ Context (programming vs corrosion) comes from ATTENTION, not the tokenizer!")

In [ ]:
# How big is the vocabulary?
print(f"Vocabulary size: {tokenizer.vocab_size:,} tokens")
print()

# Peek at some vocab entries
sample_ids = [0, 1, 2, 100, 1000, 5000, 10000, 50000, 100000]
for i in sample_ids:
    if i < tokenizer.vocab_size:
        print(f"  Token {i:>6}: '{tokenizer.decode([i])}'")

---
## Experiment 3: See the Embeddings

**Goal:** Look at REAL embedding vectors — 4,096 numbers per token (our React app showed 8 fake ones).

This is the embedding TABLE LOOKUP from our visualization: token ID → row in table → 4,096 numbers.

In [ ]:
import mlx.core as mx

# Get the embedding table — the ACTUAL weight matrix
embed_table = model.model.embed_tokens

print(f"Embedding table shape: {embed_table.weight.shape}")
print(f"  → {embed_table.weight.shape[0]:,} tokens × {embed_table.weight.shape[1]:,} dimensions")
print(f"  → Each token maps to a vector of {embed_table.weight.shape[1]:,} numbers")
print(f"  → This is the table we visualized in the React app (but 4,096 dims instead of 8)!")

In [ ]:
# Count parameters by group — match our "Inside the File" numbers

from mlx.utils import tree_flatten

def count_params(module):
    return sum(v.size for _, v in tree_flatten(module.parameters()))

embed_params = model.model.embed_tokens.weight.size
layer_params = count_params(model.model.layers[0])
total_layer_params = sum(count_params(l) for l in model.model.layers)
head_params = model.lm_head.weight.size
n_layers = len(model.model.layers)

print("Parameter Count by Group")
print("=" * 50)
print(f"Embedding Table:     {embed_params:>14,}")
print(f"Per Layer:           {layer_params:>14,}")
print(f"× {n_layers} Layers:          {total_layer_params:>14,}")
print(f"Prediction Head:     {head_params:>14,}")
print("-" * 50)
total = embed_params + total_layer_params + head_params
print(f"Total:               {total:>14,}")
print(f"\n→ This is why it's called an '8B' model!")
print(f"→ In Q4 quantization, each param ≈ 0.5 bytes → ~{total * 0.5 / 1e9:.1f} GB file")

In [ ]:
# PROOF: Same word = same embedding (before attention)
# This is what we showed in the React app!

text1 = "Rust is a programming language"
text2 = "Rust on the iron door"

ids1 = tokenizer.encode(text1)
ids2 = tokenizer.encode(text2)

emb1 = embed_table(mx.array([ids1]))
emb2 = embed_table(mx.array([ids2]))
mx.eval(emb1, emb2)

# Compare the "Rust" embedding from both sentences
rust_emb1 = emb1[0, 0, :]  # first token of sentence 1
rust_emb2 = emb2[0, 0, :]  # first token of sentence 2

diff = mx.sum(mx.abs(rust_emb1 - rust_emb2)).item()

print(f"'Rust' embedding from '{text1}'")
print(f"  First 5 values: {rust_emb1[:5].tolist()}")
print(f"\n'Rust' embedding from '{text2}'")
print(f"  First 5 values: {rust_emb2[:5].tolist()}")
print(f"\nTotal difference: {diff}")
print(f"\n→ Difference is 0.0 because embeddings are just a TABLE LOOKUP.")
print(f"→ Same token ID = same 4,096 numbers, regardless of context.")
print(f"→ The 32 attention layers AFTER this create contextual understanding.")

---
## Experiment 4: See the Weights

**Goal:** Open the model and look at the weight groups from our "Inside the File" visualization.

Remember the anatomy:
- Embedding Table
- 32 Layers (each: Attention Weights + Transform Weights)
- Prediction Head

In [ ]:
# The weight groups — exactly what we visualized!

# 1. EMBEDDING TABLE
print("1. EMBEDDING TABLE")
print(f"   Shape: {model.model.embed_tokens.weight.shape}")
print(f"   → {model.model.embed_tokens.weight.shape[0]:,} vocab × {model.model.embed_tokens.weight.shape[1]:,} dims")
print()

# 2. LAYERS
print(f"2. TRANSFORMER LAYERS: {len(model.model.layers)} layers")
layer0 = model.model.layers[0]
print(f"\n   Layer 0 — Attention Weights:")
print(f"     Q (query):  {layer0.self_attn.q_proj.weight.shape}")
print(f"     K (key):    {layer0.self_attn.k_proj.weight.shape}")
print(f"     V (value):  {layer0.self_attn.v_proj.weight.shape}")
print(f"     O (output): {layer0.self_attn.o_proj.weight.shape}")

print(f"\n   Layer 0 — Transform (Feed-Forward) Weights:")
print(f"     Gate: {layer0.mlp.gate_proj.weight.shape}")
print(f"     Up:   {layer0.mlp.up_proj.weight.shape}")
print(f"     Down: {layer0.mlp.down_proj.weight.shape}")
print()

# 3. PREDICTION HEAD
print("3. PREDICTION HEAD")
print(f"   Shape: {model.lm_head.weight.shape}")
print(f"   → Maps {model.lm_head.weight.shape[1]:,} hidden dims → {model.lm_head.weight.shape[0]:,} vocab scores")

In [ ]:
# Count parameters by group — match our "Inside the File" numbers

def count_params(module):
    return sum(p.size for p in module.parameters())

embed_params = model.model.embed_tokens.weight.size
layer_params = count_params(model.model.layers[0])
total_layer_params = sum(count_params(l) for l in model.model.layers)
head_params = model.lm_head.weight.size
n_layers = len(model.model.layers)

print("Parameter Count by Group")
print("=" * 50)
print(f"Embedding Table:     {embed_params:>14,}")
print(f"Per Layer:           {layer_params:>14,}")
print(f"× {n_layers} Layers:          {total_layer_params:>14,}")
print(f"Prediction Head:     {head_params:>14,}")
print("-" * 50)
total = embed_params + total_layer_params + head_params
print(f"Total:               {total:>14,}")
print(f"\n→ This is why it's called an '8B' model!")
print(f"→ In Q4 quantization, each param ≈ 0.5 bytes → ~{total * 0.5 / 1e9:.1f} GB file")

In [ ]:
# Peek at actual weight values
# These are the numbers that were learned during training!

q_weight = layer0.self_attn.q_proj.weight
print(f"Query weight matrix shape: {q_weight.shape}")
print(f"dtype: {q_weight.dtype}")
print(f"\nSample 5×5 corner of the Q weight matrix:")

sample = q_weight[:5, :5]
mx.eval(sample)
for row in sample.tolist():
    print(f"  [{', '.join(f'{v:+.4f}' for v in row)}]")

print(f"\n→ These specific numbers were learned from trillions of words of text.")
print(f"→ Change any one and the model behaves slightly differently.")

---
## Experiment 5: Watch Attention

**Goal:** Run text through the model and see Q, K, V — the attention mechanism we visualized.

Remember: Q = "what am I looking for?", K = "what do I contain?", V = "what info do I share?"

In [ ]:
# Get embeddings, then pass through first layer's attention

text = "Rust is a programming language"
token_ids = tokenizer.encode(text)
tokens = [tokenizer.decode([t]) for t in token_ids]
print(f"Tokens: {tokens}")

# Step 1: Embed
input_ids = mx.array([token_ids])
hidden = model.model.embed_tokens(input_ids)
print(f"After embedding: {hidden.shape}")

# Step 2: Normalize (models do this before attention)
layer0 = model.model.layers[0]
normed = layer0.input_layernorm(hidden)

# Step 3: Compute Q, K, V
q = layer0.self_attn.q_proj(normed)
k = layer0.self_attn.k_proj(normed)
v = layer0.self_attn.v_proj(normed)
mx.eval(q, k, v)

print(f"\nQ (query) shape:  {q.shape}  — 'what am I looking for?'")
print(f"K (key) shape:    {k.shape}  — 'what do I contain?'")
print(f"V (value) shape:  {v.shape}  — 'what info do I share?'")

print(f"\n→ These are the REAL Q/K/V from our attention visualization!")
print(f"→ The model multiplies Q×K to get attention scores (who looks at whom).")
print(f"→ Then uses scores to weight-sum V (gather information from attended tokens).")

In [ ]:
# Compute attention scores manually for layer 0
# This is: scores = Q × K^T / sqrt(d_k)

import math

d_k = q.shape[-1]
scores = (q @ mx.transpose(k, (0, 1, 3, 2))) / math.sqrt(d_k) if len(q.shape) == 4 else (q @ k.T) / math.sqrt(d_k)
mx.eval(scores)

print(f"Attention scores shape: {scores.shape}")
print(f"\nThese scores tell us: for each token, how much does it 'attend to' every other token?")
print(f"Higher score = pays more attention to that token.")

---
## Experiment 6: See Prediction Probabilities

**Goal:** Give the model a partial sentence and see the probability distribution — exactly like our PredictionPanel.

The model outputs a score for EVERY word in its vocabulary. Softmax turns scores → probabilities.

In [ ]:
# What comes after "Rust is a programming"?

prompt = "Rust is a programming"
token_ids = tokenizer.encode(prompt)
input_ids = mx.array([token_ids])

# Forward pass — runs through ALL layers
logits = model(input_ids)
mx.eval(logits)

# Get prediction for the LAST position
last_logits = logits[0, -1, :]  # shape: (vocab_size,)
print(f"Raw logits shape: {last_logits.shape}")
print(f"→ One score for each of {last_logits.shape[0]:,} tokens in vocabulary")

# Convert to probabilities
probs = mx.softmax(last_logits)
mx.eval(probs)

# Top 10 predictions
top_indices = mx.argsort(probs)[-10:][::-1]
mx.eval(top_indices)

print(f"\nTop 10 predictions for '{prompt} ___':\n")
for idx in top_indices.tolist():
    token_text = tokenizer.decode([idx])
    prob = probs[idx].item()
    bar = "█" * int(prob * 50)
    print(f"  {prob:6.2%}  '{token_text}'  {bar}")

In [ ]:
# Compare: confident vs uncertain predictions

prompts = [
    "Rust is a programming",         # → "language" (high confidence)
    "The capital of France is",       # → "Paris" (very high confidence)
    "I love eating",                  # → many foods (spread out)
    "Rust on the iron",               # → multiple options
    "The meaning of life is",         # → very uncertain
]

for prompt in prompts:
    ids = tokenizer.encode(prompt)
    logits = model(mx.array([ids]))
    probs = mx.softmax(logits[0, -1, :])
    top5 = mx.argsort(probs)[-5:][::-1]
    mx.eval(probs, top5)

    top1_prob = probs[top5[0]].item()
    confidence = "HIGH" if top1_prob > 0.5 else "MEDIUM" if top1_prob > 0.2 else "LOW"

    print(f"\n'{prompt} ___'  [{confidence} confidence]")
    for idx in top5.tolist():
        p = probs[idx].item()
        t = tokenizer.decode([idx])
        bar = "█" * int(p * 30)
        print(f"  {p:6.2%}  '{t}'  {bar}")

print("\n→ When confident, one token dominates (90%+).")
print("→ When uncertain, probability spreads across many tokens.")
print("→ Temperature & top-p control how we PICK from this distribution.")

---
## Experiment 7: Jarvis Model Selection

**Goal:** Test the model with Jarvis-style prompts (summarize, extract, reply).

In [ ]:
import time

# Jarvis-style tasks
jarvis_tasks = [
    {
        "name": "Summarize content",
        "prompt": "Summarize this in 3 bullet points: Transformers are a type of neural network architecture that uses self-attention mechanisms. Unlike RNNs which process sequences step by step, transformers process all tokens in parallel. The key innovation is the attention mechanism which allows each token to look at all other tokens when computing its representation. This makes transformers very efficient for training on GPUs and has led to models like GPT, BERT, and LLaMA."
    },
    {
        "name": "Draft email reply",
        "prompt": "Write a short, friendly reply to this email: 'Hey Ankit, can we reschedule our Tuesday meeting to Thursday same time? I have a conflict that came up.'"
    },
    {
        "name": "Extract key points",
        "prompt": "Extract the 3 most important facts from this: Apple's M3 Pro chip features an 11-core CPU with 5 performance and 6 efficiency cores. It has 18-core GPU and supports up to 36GB of unified memory. The chip is built on 3nm process technology and delivers up to 40% faster CPU performance compared to M1 Pro."
    },
    {
        "name": "Explain code",
        "prompt": "Explain what this Rust code does in plain English: fn fibonacci(n: u32) -> u32 { match n { 0 => 0, 1 => 1, _ => fibonacci(n-1) + fibonacci(n-2) } }"
    }
]

for task in jarvis_tasks:
    print(f"\n{'=' * 60}")
    print(f"TASK: {task['name']}")
    print(f"{'=' * 60}")
    
    start = time.time()
    response = generate(model, tokenizer, prompt=task['prompt'], max_tokens=200, verbose=True)
    elapsed = time.time() - start
    
    print(response)
    print(f"\n⏱  {elapsed:.1f}s")

In [ ]:
# Speed benchmark
import time

prompt = "Write a detailed paragraph about the benefits of local AI models for privacy and performance."

start = time.time()
response = generate(model, tokenizer, prompt=prompt, max_tokens=300, verbose=True)
elapsed = time.time() - start

# Count output tokens
output_tokens = len(tokenizer.encode(response))

print(f"\n{'=' * 40}")
print(f"Output tokens: {output_tokens}")
print(f"Total time: {elapsed:.1f}s")
print(f"Speed: {output_tokens/elapsed:.1f} tokens/sec")
print(f"\n→ For Jarvis: a typical response (50-100 tokens) takes {50/max(output_tokens/elapsed, 1):.1f}-{100/max(output_tokens/elapsed, 1):.1f}s")

---
## Experiment 8: Quantization — See the Trade-off

**Goal:** Understand why Q4 works. Each weight stored in 4 bits instead of 16.

In [ ]:
import os

# Check memory usage
pid = os.getpid()

# Model info
total_params = sum(p.size for p in model.parameters())
print(f"Total parameters: {total_params:,}")
print(f"\nIf stored at full precision (float32 = 4 bytes each):")
print(f"  {total_params * 4 / 1e9:.1f} GB")
print(f"\nIf stored at half precision (float16 = 2 bytes each):")
print(f"  {total_params * 2 / 1e9:.1f} GB")
print(f"\nAt Q4 quantization (4 bits = 0.5 bytes each):")
print(f"  {total_params * 0.5 / 1e9:.1f} GB")
print(f"\n→ Quantization shrinks the model ~4× with minimal quality loss.")
print(f"→ That's why a 8B model fits comfortably on your 36 GB Mac!")

In [ ]:
# Look at quantized weight dtype
layer0 = model.model.layers[0]
q_weight = layer0.self_attn.q_proj.weight

print(f"Weight dtype: {q_weight.dtype}")
print(f"Weight shape: {q_weight.shape}")
print(f"\n→ The values are packed in 4-bit format.")
print(f"→ During inference, they get unpacked to float16/float32 for computation.")
print(f"→ Small precision loss, big memory savings.")

---
## Summary: What Connects to Our React App

| React App (mock data) | MLX Experiment (real data) |
|---|---|
| TokenizerPanel (simple split) | Exp 2: Real BPE tokenizer with 151k vocab |
| EmbeddingPanel (8 fake dims) | Exp 3: Real 4,096-dim embeddings |
| "Inside the File" (diagrams) | Exp 4: Actual weight shapes and counts |
| AttentionPanel (mock scores) | Exp 5: Real Q/K/V matrices |
| PredictionPanel (fake top-5) | Exp 6: Real probability distribution |
| AutoregressiveDemo | Exp 1: Watch real token-by-token generation |
| KV Cache Demo (theoretical) | Exp 1: Actual tokens/sec (cache is built-in) |

**Next steps after these experiments:**
1. Integrate Qwen 8B into Jarvis via MLX-LM Python API
2. Build Part 3 of React app (Model Sizes, Quantization, Memory) using real data from these experiments
3. Explore fine-tuning with LoRA (teach the model Jarvis-specific behavior)